<a href="https://colab.research.google.com/github/Atharv-16/GSOC/blob/main/Current-Progress/dino_with_equiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download and extracting dataset

In [1]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 1aafE2nDp7S6j59sZcBIzP3FnQxVCmHCx
!unzip real_hst.zip

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1aafE2nDp7S6j59sZcBIzP3FnQxVCmHCx
From (redirected): https://drive.google.com/uc?id=1aafE2nDp7S6j59sZcBIzP3FnQxVCmHCx&confirm=t&uuid=30ba8252-ac2c-41a9-b2af-fe17fa1f95b9
To: /content/real_hst.zip
100% 80.5M/80.5M [00:00<00:00, 167MB/s]
Archive:  real_hst.zip
   creating: real_hst/
  inflating: real_hst/nl_269_hst.npy  
  inflating: real_hst/75_hst.npy     
  inflating: real_hst/65_hst.npy     
  inflating: real_hst/nl_301_hst.npy  
  inflating: real_hst/9372_hst.npy   
  inflating: real_hst/17_hst.npy     
  inflating: real_hst/nl_189_hst.npy  
  inflating: real

# Downloading Equivariant Network Source Codes

In [2]:
!git clone https://github.com/ML4SCI/DeepLense.git

Cloning into 'DeepLense'...
remote: Enumerating objects: 2217, done.
remote: Counting objects: 100% (551/551), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 2217 (delta 335), reused 423 (delta 261), pack-reused 1666
Receiving objects: 100% (2217/2217), 535.78 MiB | 26.84 MiB/s, done.
Resolving deltas: 100% (1040/1040), done.
Updating files: 100% (823/823), done.


**Important Instruction**: You will have to change line 74 to **return F.grid_sample(x, grid)** and comment from 75 to 82 in DeepLense/Equivariant_Neural_Networks_for_DeepLense_GEO/models/equivariant_transformers/network.py and then restart runtime

> Add blockquote



In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Input and output directories
input_folder = "./real_hst"
output_folder = "./output_folder"

# Create output directories for classes
nl_folder = os.path.join(output_folder, "nl")
others_folder = os.path.join(output_folder, "others")
os.makedirs(nl_folder, exist_ok=True)
os.makedirs(others_folder, exist_ok=True)

# Iterate through files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".npy"):
        filepath = os.path.join(input_folder, filename)
        # Load the numpy array
        data = np.load(filepath)
        # Determine class based on filename
        if filename.startswith("nl"):
            class_folder = nl_folder
        else:
            class_folder = others_folder
        # Convert array to image and save
        img_path = os.path.join(class_folder, filename.replace(".npy", ".png"))
        plt.imsave(img_path, data, cmap='gray')  # Adjust colormap as needed


# Importing Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random

# Seed

In [3]:
def set_seed(no):
    torch.manual_seed(no)
    random.seed(no)
    np.random.seed(no)
    os.environ['PYTHONHASHSEED'] = str()
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_seed(100)

In [4]:
print(torch.__version__)

2.2.1+cu121


# Data preprocessing and data loaders

In [5]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Define data transformations
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Set the directory containing your data
data_dir = '/content/output_folder'

# Use ImageFolder to load data
image_dataset = datasets.ImageFolder(data_dir, transform=data_transforms)

# Split the dataset into train and test sets (80% train, 20% test)
train_size = int(0.8 * len(image_dataset))
test_size = len(image_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(image_dataset, [train_size, test_size])

# Create data loaders
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=6, shuffle=True, num_workers=4),
    'test': DataLoader(test_dataset, batch_size=6, shuffle=False, num_workers=4)
}

# Get class names
class_names = image_dataset.classes

# Get dataset sizes
dataset_sizes = {'train': len(train_dataset), 'test': len(test_dataset)}

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [6]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7e1cd66476a0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7e1cd66470a0>}

In [7]:
# !cd /content/DeepLense/Equivariant_Neural_Networks_for_DeepLense_GEO
!pip install e2cnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.3/225.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# DINO Model


In [8]:
# load dino model
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 119MB/s]


In [9]:
%cd /content/DeepLense/Equivariant_Neural_Networks_for_DeepLense_GEO/

/content/DeepLense/Equivariant_Neural_Networks_for_DeepLense_GEO


# Initializing Equivariant Transformer

In [10]:
from models import coordinates, networks, transformers, ET

tf_default_opts = {
    "in_channels": 3,  # Output channels of DINO model
    "kernel_size": 3,
    "nf": 32,
    "strides": (2, 1),
}

et_default_opts = {
    "input_channels": 3,  # Output channels of DINO model
    "output_size": 3,  # Number of classes
    "nf": 32,
    "p_dropout": 0.3,
    "pad_mode": (None, "cyclic"),
    "pool": (True, True, False),
}

ET_ = ET(
    tfs=[
        transformers.ShearX,
        transformers.HyperbolicRotation,
        transformers.PerspectiveX,
        transformers.PerspectiveY,
    ],
    coords=coordinates.logpolar_grid,
    net=networks.BasicCNN,
    equivariant=True,
    tf_opts=tf_default_opts,
    net_opts=et_default_opts,
)

# Training Parameters

In [12]:
from tqdm import tqdm
import gc
lr = 0.0001
epochs = 30
gamma = 0.5
batch_size = 64
device = "cuda" if torch.cuda.is_available() else "cpu"
ET_.model = ET_.model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(ET_.model.parameters(), lr=0.00001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=gamma)

# Training

In [15]:
all_train_loss = []
all_test_loss = []
all_train_accuracy = []
all_test_accuracy = []

best_accuracy = 0

for epoch in range(3):
    epoch_loss = 0
    epoch_accuracy = 0
    correct, total = 0, 0
    ET_.model.train()
    tr_loss_epoch = []
    test_loss_epoch = []
    _ep = 0
    for data, label in tqdm(dataloaders['train']):
        # print(data.shape)
        if _ep%100 == 0:
            torch.cuda.empty_cache()
            gc.collect()
        data = data.to(device)
        label = label.to(device)
        output = ET_.model(data)
        output=dinov2_vits14(output)
        output=nn.Sequential(
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Linear(256, 2))(output)

        loss = criterion(output, label)
        tr_loss_epoch.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == label).sum()
        correct += acc.item()
        total += len(label)

        epoch_loss += loss.item() / len(dataloaders['train'])
        _ep += 1
    scheduler.step()
    all_train_loss.append(np.asarray(tr_loss_epoch))
    all_train_accuracy.append(correct / total * 100)
    epoch_accuracy = (correct / total) * 100
    correct, total = 0, 0

    torch.cuda.empty_cache()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in tqdm(dataloaders['test']):
            data = data.to(device)
            label = label.to(device)
            output = ET_.model(data)
            output=dinov2_vits14(output)
            val_output=nn.Sequential(
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Linear(256, 2))(output)
            # print("val",val_output.shape)
            # print(val_output)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f}% - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}%"
    )
ET_._save("equi_nn.pth")
all_train_loss_mean = [j.mean() for j in all_train_loss]
all_test_loss_mean = [j.mean() for j in all_test_loss]

100%|██████████| 8/8 [00:14<00:00,  1.85s/it]


Epoch : 1 - loss : 0.7060 - acc: 55.2326% - val_loss : 0.0000 - val_acc: 0.0000%


100%|██████████| 8/8 [00:14<00:00,  1.83s/it]


Epoch : 2 - loss : 0.7147 - acc: 56.3953% - val_loss : 0.0000 - val_acc: 0.0000%


100%|██████████| 8/8 [00:14<00:00,  1.84s/it]

Epoch : 3 - loss : 0.7806 - acc: 50.5814% - val_loss : 0.0000 - val_acc: 0.0000%


# Testing

In [18]:
# correct = 0
# total = 0
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     for data in dataloaders["test"]:
#         images, labels = data
#         # calculate outputs by running images through the network
#         outputs = model(images.to(device))
#         # the class with the highest energy is what we choose as prediction
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted.to("cpu") == labels).sum().item()

# print(f'Accuracy of the network on the {len(dataloaders["test"])*6} test images: {100 * correct // total} %')